In [5]:
from langchain_ollama import OllamaLLM

In [8]:
!ollama list

NAME            ID              SIZE      MODIFIED     
gemma3:12b      f4031aab637d    8.1 GB    2 months ago    
gemma3:4b       a2af6cc3eb7f    3.3 GB    2 months ago    
llava:latest    8dd30f6b0cb1    4.7 GB    2 months ago    


In [32]:
# Initialize the local model
llm = OllamaLLM(model="gemma3:4b")

print(llm.invoke("capital of france?"))

The capital of France is **Paris**. 

It’s a global center for art, fashion, gastronomy, and culture. 😊 

Do you want to know anything more about Paris or France?


In [ ]:
import wikipedia

def get_wikipedia_summary(query: str) -> str:
    try:
        return wikipedia.summary(query, sentences=5)
    except wikipedia.exceptions.DisambiguationError as e:
        return f"Too many results. Try one of these: {e.options[:5]}"
    except Exception as e:
        return f"Error: {str(e)}"

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

template = """
You are a helpful AI assistant.
Based on the following Wikipedia content, answer the user's question.

Wikipedia content:
{context}

Question:
{question}

Answer:
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template
)

chain = LLMChain(llm=llm, prompt=prompt)



def wikipedia_qa(question: str):
    context = get_wikipedia_summary(question)
    answer = chain.run({"context": context, "question": question})
    return answer


while True:
    q = input("Ask a question (or 'quit'): ")
    if q.lower() == "quit":
        break
    print("Answer:", wikipedia_qa(q))


In [115]:
wikipedia_search("lol")

'LOL, or lol, is an initialism for laughing out loud, and a popular element of Internet slang, which can be used to indicate amusement, irony, or double meanings. It was first used almost exclusively on Usenet, but has since become widespread in other forms of computer-mediated communication and even face-to-face communication. It is one of many initialisms for expressing bodily reactions, in particular laughter, as text, including initialisms for more emphatic expressions of laughter such as LMAO ("laughing my ass off") and ROFL or ROTFL ("rolling on the floor laughing").\nIn 2003, the list of acronyms was said to "grow by the month", and they were collected along with emoticons and smileys into folk dictionaries that are circulated informally amongst users of Usenet, IRC, and other forms of (textual) computer-mediated communication. These initialisms are controversial, and several authors recommend against their use, either in general or in specific contexts such as business communic

In [52]:
print(wikipedia_search('tell me about korea before it splitted'))

Error: Page id "tell me about korean before it spotted" does not match any pages. Try another id!


In [116]:
# Initialize the local model
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_ollama import OllamaLLM  # Or use OpenAI
import wikipedia 

llm_ollama = OllamaLLM(model="gemma3:4b")


def wikipedia_search(query: str , num_sentences:int = 5) ->str:
    """
    Docstring:   Searches wikipedia homepage given a word.
    
    args:
        query (str): the query for wikipedia.
        num_sentences  (int): number of output sentences.

    returns:
        result (str): the result from wikipedia.

    """
    try:
        most_close_query = wikipedia.search(query)[0]

        summary = wikipedia.page(most_close_query , auto_suggest=False, redirect=True).summary
        sentences = summary.split('. ')
        return '. '.join(sentences[:num_sentences]) + '.'
        #return summary
        
    except wikipedia.exceptions.DisambiguationError as e:
        return f"Too many results please choose from following topics: {e.options[0:6]}" 
    except Exception as e:
        return f"Error: {str(e)}"



def ask_llm(question:str  ) -> str:
    """
    This function will use ollama for converting the question into a valid wikipedia search keyword.

    Args:
        question (str): The entered question. 
    
    Returns:
        result (str): The valid wikipedia keyword
    """
    prompt = PromptTemplate.from_template(
        "Given the following question, output only the most relevant Wikipedia search keyword or page title, and nothing else.\n\nQuestion: {question}\n\nKeyword:"
    )

    chain = LLMChain(llm=llm_ollama, prompt=prompt)
    return chain.run(question) 

def ask_llm_with_context(question:str , wikipedia_context:str) -> str:
    """
    This function will take the question and context and will gives the answer to the question.

    Args:
        question (str): The entered question
        context (str): The summary from wikipedia page
    Returns:
        result (str): The full answer to the question
    """

    prompt = PromptTemplate.from_template(
        "You are a helpful assistant, answer the question based on context. \n\n Question: {question} \n\n Context: {context}"
    )
    chain = LLMChain(llm = llm_ollama , prompt = prompt)
    return chain.run({"context": wikipedia_context, "question": question})

def run_bot(question):
    
    topic_for_wikipedia = ask_llm(question) 
    context = wikipedia_search(topic_for_wikipedia)
    return ask_llm_with_context(question , context)

if __name__ == "__main__":
    print(r"Welcome to wikipedia QA bot. Type [\exit , \stop ,\q , \s] to stop the bot")
    while(True):
        question = str(input("Ask a question:"))
        if question.lower() in [r"\exit", r"\stop" , r"\stopp" , r"\quit" , r"\quitt" , r"\q"]: 
            print("Goodbye! :) ")
            break

        print(run_bot(question) , "\n\n")
        

Welcome to wikipedia QA bot. Type [\exit , \stop ,\q , \s] to stop the bot


Ask a question: lol


The question "lol" is an internet slang abbreviation for "laugh out loud." 




Ask a question: \q


Goodbye! :) 


In [97]:
query = wikipedia.search("Augusto Pinochet")[0]

wikipedia.page(query , auto_suggest=False, redirect=True).summary

'Augusto José Ramón Pinochet Ugarte (25 November 1915 – 10 December 2006) was a Chilean military officer and politician who was the dictator of Chile from 1973 to 1990. From 1973 to 1981, he was the leader of the military junta, which in 1974 declared him President of the Republic and thus the dictator of Chile; in 1980, a referendum approved a new constitution confirming him in the office, after which he served as de jure president from 1981 to 1990. His time in office remains the longest of any Chilean ruler.\nAugusto Pinochet rose through the ranks of the Chilean Army to become General Chief of Staff in early 1972 before being appointed its Commander-in-Chief on 23 August 1973 by President Salvador Allende. On 11 September 1973, \nPinochet seized power in Chile in a military coup. The military had previously received financial and intelligence support from the United States, which favored the military coup that toppled Allende\'s democratically elected left-wing Unidad Popular gover

In [103]:
wikipedia_search("Augusto Pinochet")

"Error: 'str' object is not callable"